In [1]:

%pprint
import sys
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))
from FRVRS import (nu, fu, DataFrame, np, nan, Series)
import pandas as pd

Pretty printing has been turned OFF



# Find the 389

In [2]:

# Download the Oct 24, 2023 version of frvrs_logs_df.csv from the Data and Videos page on Confluence
frvrs_logs_df = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df')['frvrs_logs_df']
print(frvrs_logs_df.shape) # (829277, 113)

No pickle exists for frvrs_logs_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv.
(829277, 113)


In [3]:

# Review any missing columns
needed_set = set(['session_uuid', 'scene_id', 'scene_type', 'is_scene_aborted', 'file_name'])
for cn in needed_set:
    if cn not in frvrs_logs_df.columns: print(f'The {cn} column is not in the frvrs_logs_df DataFrame.')

The scene_id column is not in the frvrs_logs_df DataFrame.


In [4]:

# Scene index was changed to scene ID on December 14, 2023
if 'scene_id' not in frvrs_logs_df.columns: frvrs_logs_df = frvrs_logs_df.rename(columns={'scene_index': 'scene_id'})

In [5]:

# Potentially add missing columns
if not all(map(lambda cn: cn in frvrs_logs_df.columns, ['scene_type', 'is_scene_aborted'])):
    for (session_uuid, scene_id), scene_df in frvrs_logs_df.groupby(fu.scene_groupby_columns):
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        
        # Scene type added November 29, 2023
        if 'scene_type' not in frvrs_logs_df.columns: frvrs_logs_df.loc[mask_series, 'scene_type'] = fu.get_scene_type(scene_df)
        
        # Is scene aborted added November 14, 2023
        if 'is_scene_aborted' not in frvrs_logs_df.columns: frvrs_logs_df.loc[mask_series, 'is_scene_aborted'] = fu.get_is_scene_aborted(scene_df)

In [6]:

# Verify that we found it
path_prefix = 'Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/'
original_files_list = [
    file_name.replace(path_prefix, '') for file_name, file_name_df in frvrs_logs_df.groupby('file_name') if fu.get_is_a_one_triage_file(file_name_df)
]
original_files_count = len(original_files_list)
assert 389 == original_files_count, "This CSV doesn't contain the 389"

In [7]:

# Create a session UUIDs list
mask_series = frvrs_logs_df.file_name.isin([path_prefix + f for f in original_files_list]) & ~frvrs_logs_df.session_uuid.isnull()
original_session_uuids_list = sorted(frvrs_logs_df[mask_series].session_uuid.unique())
original_session_uuids_count = len(original_session_uuids_list)


# Identify any Anomalous Files

In [8]:

anomalous_files_set = set()
anomalous_files_str = ''
explanations_list = []

# Get a dataset of the original one-triage files
one_triage_files_df = frvrs_logs_df.groupby('file_name').filter(lambda file_name_df: fu.get_is_a_one_triage_file(file_name_df))

In [9]:

# If all the patients in a file are all named Mike, that is a training simulation
files_list = []

# Group the Data Frame by 'file_name'
for file_name, file_name_df in one_triage_files_df.groupby('file_name'):

    # Get a list of the unique patient IDs in the file
    patient_ids = file_name_df.patient_id.unique().tolist()

    # Check if all the patient IDs in the file contain the string "mike" in lowercase
    if all(map(lambda x: 'mike' in str(x).lower(), patient_ids)):

        # Add the file name to the files_list list
        files_list.append(file_name)

# If there are files with all patients named "Mike," print and update the results
if files_list:

    # Create a string with the list of files having "Mike" patients
    print_str = f'\n\nThese {len(files_list)} files have patients that are all named "Mike":\n\t{nu.conjunctify_nouns(files_list)}'
    explanations_list.append(f'{len(files_list)} files had patients that are all named "Mike"')

    # Add carriage returns for better readability
    print_str = print_str.replace(' and ', ', and\n\t')
    
    # Print the results
    print(print_str)

    # Update the anomalous files string
    anomalous_files_str += print_str

    # Update the anomalous files set
    anomalous_files_set.update(files_list)

In [10]:

# Initialize an empty list to store rows
rows_list = []

# Loop through sessions and scenes
for (session_uuid, scene_id), scene_df in one_triage_files_df.groupby(fu.scene_groupby_columns):

    # Create a dictionary to store the results for the current group
    row_dict = {}

    # Get the logger version for the current group
    logger_version = fu.get_logger_version(scene_df)

    # Add the logger version, session UUID, and scene to the dictionary
    for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)
    row_dict['logger_version'] = logger_version

    # Count the number of user actions for the current group
    row_dict['total_actions'] = fu.get_total_actions_count(scene_df)

    # Append the row to the list
    rows_list.append(row_dict)

# Create a data frame from the list of rows
total_actions_df = DataFrame(rows_list)

# Group by session UUID and sum total actions
df = total_actions_df.groupby('session_uuid').sum()

# Create a mask for sessions with zero total actions
mask_series = (df.total_actions == 0)

# Get a list of session UUIDs with no user actions
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter for UUIDs with no user actions
mask_series = one_triage_files_df.session_uuid.isin(session_uuids_list)

# Get unique file names associated with sessions with zero total actions
files_list = one_triage_files_df[mask_series].file_name.unique().tolist()

# Check if there are files with zero actions
if files_list:
    
    # Create a formatted string of anomalous files
    print_str = f'\n\nThese {len(files_list)} files have no user action taken:\n\t{nu.conjunctify_nouns(files_list)}'
    explanations_list.append(f'{len(files_list)} files had no user action taken')
    
    # Add carriage returns for readability
    print_str = print_str.replace(' and ', ', and\n\t')

    # Print and update the anomalous files string and set
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These 2 files have no user action taken:
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/7c2549d4-97a4-4389-bd03-029396714f59.csv, and
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.0/Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv


In [11]:

# Create a mask to filter rows where action_type is PULSE_TAKEN or INJURY_TREATED
mask_series = one_triage_files_df.action_type.isin(['PULSE_TAKEN', 'INJURY_TREATED'])

# Define columns to group by and initialize lists


rows_list = []
indices_list = []

# Iterate over groups based on the specified columns
for (session_uuid, scene_id, patient_id), patient_df in one_triage_files_df[mask_series].groupby(fu.patient_groupby_columns):

    # Create a tuple to store the index of the current row
    index_tuple = (session_uuid, scene_id, patient_id)
    indices_list.append(index_tuple)

    # Create a dictionary to store the results for the current row
    row_dict = {}

    # Add the logger_version for the current row
    row_dict['logger_version'] = fu.get_logger_version(patient_df)
    
    # Count the number of 'PULSE_TAKEN' actions in this group
    row_dict['pulses_count'] = fu.get_pulse_taken_count(patient_df)
    
    # Count the number of 'INJURY_TREATED' actions in this group
    row_dict['treated_count'] = fu.get_injury_treatments_count(patient_df)
    
    # Try to calculate the number of pulses taken per injury treated
    try: row_dict['pulses_by_treated'] = row_dict['pulses_count'] / row_dict['treated_count']

    # Handle the case where 'treated_count' is zero to avoid division by zero
    except ZeroDivisionError: row_dict['pulses_by_treated'] = nan
    
    # Add the row_dict to the rows_list
    rows_list.append(row_dict)

# Create a data frame from the rows and set a multi index based on the grouped columns
pulses_count_df = DataFrame(rows_list, index=pd.MultiIndex.from_tuples(tuples=indices_list, names=fu.patient_groupby_columns))

# Group the pulses count data frame by session_uuid and sum the values
df = pulses_count_df.groupby('session_uuid').sum()

# Check for sessions with 'treated_count' equal to zero
mask_series = (df.treated_count == 0)
session_uuids_list = df[mask_series].index.tolist()

# Filter logs for sessions with no 'INJURY_TREATED' actions
mask_series = one_triage_files_df.session_uuid.isin(session_uuids_list)
files_list = one_triage_files_df[mask_series].file_name.unique().tolist()

# Print files with no injury treatment being done
if files_list:
    print_str = f'\n\nThese {len(files_list)} files have no injury treatment being done:\n\t{nu.conjunctify_nouns(files_list)}'
    explanations_list.append(f'{len(files_list)} files had no injury treatment being done')
    print_str = print_str.replace(' and ', ', and\n\t')
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)

# Check for sessions with 'pulses_count' equal to zero
mask_series = (df.pulses_count == 0)
session_uuids_list = df[mask_series].index.tolist()

# Filter logs for sessions with no 'PULSE_TAKEN' actions
mask_series = one_triage_files_df.session_uuid.isin(session_uuids_list)
files_list = one_triage_files_df[mask_series].file_name.unique().tolist()

# Print files with no pulses being taken
if files_list:
    print_str = f'\n\nThese {len(files_list)} files have no pulses being taken:\n\t{nu.conjunctify_nouns(files_list)}'
    explanations_list.append(f'{len(files_list)} files had no pulses being taken')
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These 5 files have no injury treatment being done:
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/All CSV files renamed by date/03.14.23.0919.csv, Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/All CSV files renamed by date/11.30.20.0828.csv, Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/1066671d-2a1d-4744-b66f-e4b48548701f.csv, Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/54aaf31a-22bc-46f2-a810-8564161bf8d0.csv,, and
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.0/Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv


These 6 files have no pulses being taken:
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/All CSV files renamed by date/03.15.23.0944.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/0786a1df-d010-4b1b-a99a-e00df

In [12]:

# Patient accuracy rate (how many patients correct / number of patients treated)

# Initialize an empty list to store dictionaries
rows_list = []

# Iterate over the scenes
for (session_uuid, scene_id), scene_df in one_triage_files_df.groupby(fu.scene_groupby_columns):

    # Get the logger version for the group
    logger_version = fu.get_logger_version(scene_df)
    
    # Create a dictionary to store the results for the group
    row_dict = {}
    for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)
    row_dict['logger_version'] = logger_version

    # Get the total number of patients treated
    total_treated = fu.get_injury_treatments_count(scene_df)

    # If there were any patients treated, calculate the patient accuracy rate
    if total_treated:
        
        # Filter the dataframe to only include rows where the injury was treated correctly
        correctly_treated = fu.get_injury_correctly_treated_count(scene_df)

        # Calculate the patient accuracy rate
        row_dict['injury_treated_total_treated'] = total_treated
        row_dict['injury_treated_correctly_treated'] = correctly_treated
        row_dict['injury_treated_patient_accuracy_rate'] = correctly_treated / total_treated
    
    # Filter the data frame to only include rows where the injury record was treated.
    total_mask = (scene_df.injury_record_injury_treated == True)
    df2 = scene_df[total_mask]

    # Get the total number of patients treated
    total_treated = df2.shape[0]

    # If there were any patients treated, calculate the patient accuracy rate
    if total_treated:
        
        # Filter the dataframe to only include rows where the injury record was treated correctly
        correct_mask = (df2.injury_record_injury_treated_with_wrong_treatment == False)
        correctly_treated = df2[correct_mask].shape[0]

        # Calculate the patient accuracy rate
        row_dict['injury_record_total_treated'] = total_treated
        row_dict['injury_record_correctly_treated'] = correctly_treated
        row_dict['injury_record_patient_accuracy_rate'] = correctly_treated / total_treated

    # Add the row dictionary to the list of results
    rows_list.append(row_dict)

# Create a data frame from the list of dictionaries
patient_accuracy_rate_df = DataFrame(rows_list)

# Modalize into one patient accuracy rate column if possible
if 'injury_record_patient_accuracy_rate' in patient_accuracy_rate_df.columns:

    # Get a list of columns that contain the patient accuracy rate
    columns_list = [
        'injury_treated_patient_accuracy_rate', 'injury_record_patient_accuracy_rate'
    ]
    
    # Check if there's only one unique value across the specified columns
    mask_series = (patient_accuracy_rate_df[columns_list].apply(Series.nunique, axis='columns') == 1)
    
    # Set the patient accuracy rate column for the rows identified by the mask series to the non-null value in one of the patient accuracy rate columns
    patient_accuracy_rate_df.loc[~mask_series, 'patient_accuracy_rate'] = nan
    
    # Define a function to select the first valid value
    def f(srs):
        cn = srs.first_valid_index()
        
        return srs[cn]
    
    # Modalize the patient accuracy rate columns to get a single column
    patient_accuracy_rate = patient_accuracy_rate_df[mask_series][columns_list].apply(f, axis='columns')
    
    # Set the patient accuracy rate column for the rows identified by the mask series to the single patient accuracy rate column
    patient_accuracy_rate_df.loc[mask_series, 'patient_accuracy_rate'] = patient_accuracy_rate
    
    # Group by 'session_uuid' and sum the patient accuracy rates
    df = patient_accuracy_rate_df.groupby('session_uuid').sum()
    mask_series = (df.patient_accuracy_rate == 0)
else:
    
    # Group by 'session_uuid' and sum the patient accuracy rates for 'injury_treated' data
    df = patient_accuracy_rate_df.groupby('session_uuid').sum()
    mask_series = (df.injury_treated_patient_accuracy_rate == 0)

# Get a list of session_uuids with no patient accuracy rate above zero
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter rows in 'one_triage_files_df' based on session_uuids
mask_series = one_triage_files_df.session_uuid.isin(session_uuids_list)

# Get a list of unique file names from the filtered rows
files_list = one_triage_files_df[mask_series].file_name.unique().tolist()

# Print the list of files with no patient accuracy rate above zero
if files_list:
    print_str = f'\n\nThese {len(files_list)} files have no patient accuracy rate above zero:\n\t{nu.conjunctify_nouns(files_list)}'
    explanations_list.append(f'{len(files_list)} files had no patient accuracy rate above zero')
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These 13 files have no patient accuracy rate above zero:
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/All CSV files renamed by date/03.14.23.0919.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/All CSV files renamed by date/11.30.20.0828.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/All CSV files renamed by date/12.06.22.1331.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/1066671d-2a1d-4744-b66f-e4b48548701f.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/54aaf31a-22bc-46f2-a810-8564161bf8d0.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/7c2549d4-97a4-4389-bd03-029396714f59.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/8ec8afba-8533-4915-898f-5769c1258c61.csv,
	Us

In [13]:

# Number of patients engaged

# Initialize an empty list to store rows
rows_list = []

# Iterate over the sessions, grouped by scene
for (session_uuid, scene_id), scene_df in one_triage_files_df.groupby(fu.scene_groupby_columns):

    # Get the logger version
    logger_version = fu.get_logger_version(scene_df)

    # Get the number of patients in the session
    patients_count = fu.get_patient_count(scene_df)

    # Initialize the number of patients engaged
    patients_engaged = 0
    
    # Loop over patients within this scene
    for patient_id, patient_df in scene_df.groupby('patient_id'):
        
        # Create a mask to filter rows where action_type is 'PATIENT_ENGAGED'
        mask_series = (patient_df.action_type == 'PATIENT_ENGAGED')
        
        # If there are any rows that match the mask, increment the number of patients engaged
        if mask_series.any(): patients_engaged += 1

    # Check if there are patients in this scene
    if patients_count:
        # Create a dictionary to store row data
        row_dict = {}

        # Add the session UUID and the scene to the dictionary
        for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)

        # Add the logger version to the dictionary
        row_dict['logger_version'] = logger_version

        # Add the number of patients to the dictionary
        row_dict['patients_count'] = patients_count

        # Add the number of patients engaged to the dictionary
        row_dict['patients_engaged'] = patients_engaged

        # Calculate the percentage of patients engaged
        row_dict['percentage_engaged'] = patients_engaged / patients_count

        # Add the row dictionary to the list of rows
        rows_list.append(row_dict)

# Create a data frame from the list of rows
percentage_engaged_df = DataFrame(rows_list)

# Group the Data Frame by UUID and sum the number of patients engaged
df = percentage_engaged_df.groupby('session_uuid').sum()

# Create a mask to identify UUIDs with no engaged patients
mask_series = (df.patients_engaged == 0)

# List the UUIDs that meet the condition
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter rows in the main data frame based on UUID
mask_series = one_triage_files_df.session_uuid.isin(session_uuids_list)

# Get unique file_names that match the UUIDs
files_list = one_triage_files_df[mask_series].file_name.unique().tolist()

# Check if there are any files with no patients engaged
if files_list:

    # Create a formatted string to list files with no engaged patients
    print_str = f'\n\nThese {len(files_list)} files have no patients being engaged:\n\t{nu.conjunctify_nouns(files_list)}'
    explanations_list.append(f'{len(files_list)} files had no patients being engaged')
    
    # Format the string to have line breaks after commas
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')

    # Print the results and add the results to the anomalous_files_str string
    print(print_str); anomalous_files_str += print_str

    # Add the files to the anomalous_files_set set
    anomalous_files_set.update(files_list)



These 8 files have no patients being engaged:
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/1066671d-2a1d-4744-b66f-e4b48548701f.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/54aaf31a-22bc-46f2-a810-8564161bf8d0.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/7c2549d4-97a4-4389-bd03-029396714f59.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/DCEMS Round 2 only triage sessions/ccfb5502-64b5-4d1c-9ca3-63f522330041.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.0/Clean 158e6365-673b-4030-8b36-6704be5996a2.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.0/Clean 2310f107-d9d2-418e-a2d7-dd7a17924544.csv,
	Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.0/Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv,

In [14]:

anomalous_files_list = [f.replace(path_prefix, '') for f in anomalous_files_set]
anomalous_files_count = len(anomalous_files_list)
# print(f"\nDoug,\n\nHere is the set of {anomalous_files_count} anomalous files I'm concerned about:")
# print('\t' + '\n\t'.join(sorted(anomalous_files_list)))
# print(anomalous_files_str)

In [15]:

# Create a session UUIDs list
mask_series = one_triage_files_df.file_name.isin([path_prefix + f for f in anomalous_files_list]) & ~one_triage_files_df.session_uuid.isnull()
anomalous_session_uuids_list = sorted(one_triage_files_df[mask_series].session_uuid.unique())
anomalous_session_uuids_count = len(anomalous_session_uuids_list)

In [17]:

# load triage paper data frame and get the maximum number of files to compare
data_frames_dict = nu.load_data_frames(
    first_responder_master_registry_df='', first_responder_master_registry_file_stats_df='', first_responder_master_registry_scene_stats_df='',
    verbose=False
)
triage_paper_df = data_frames_dict['first_responder_master_registry_df']
file_stats_df = data_frames_dict['first_responder_master_registry_file_stats_df']
scene_stats_df = data_frames_dict['first_responder_master_registry_scene_stats_df']

# Merge the registry data frames to get the original columns
patient_count_filter_fn = lambda scene_df: True
merge_df = fu.get_elevens_dataframe(
    triage_paper_df, file_stats_df, scene_stats_df,
    needed_columns=needed_set,
    patient_count_filter_fn=patient_count_filter_fn
)

# Show how many one-triage files there are
registry_files_list = [file_name for file_name, file_name_df in merge_df.groupby('file_name') if fu.get_is_a_one_triage_file(file_name_df)]
registry_files_count = len(registry_files_list)
# assert 389 == registry_files_count, "This dataset doesn't contain the 389"

In [18]:

# Create a session UUIDs list
mask_series = merge_df.file_name.isin(registry_files_list) & ~merge_df.session_uuid.isnull()
registry_session_uuids_list = sorted(merge_df[mask_series].session_uuid.unique())
registry_session_uuids_count = len(registry_session_uuids_list)

In [19]:

# Show what we still have to account for
original_session_uuids_count - registry_session_uuids_count - anomalous_session_uuids_count

42

In [20]:

len(set(original_session_uuids_list) - set(registry_session_uuids_list) - set(anomalous_session_uuids_list))

48

In [21]:

original_session_uuids_list[0], registry_session_uuids_list[0], anomalous_session_uuids_list[0]

('009b848c-ea64-4f22-bd40-711213a3d327', '009b848c-ea64-4f22-bd40-711213a3d327', '0786a1df-d010-4b1b-a99a-e00df486d479')

In [26]:

# Assert that the anomalous sessions were taken from the original sessions
assert original_session_uuids_count - anomalous_session_uuids_count == len(set(original_session_uuids_list) - set(anomalous_session_uuids_list))

In [24]:

# Show what we still have to account for
anomalous_session_uuids_count

19

In [27]:

# Show the session UUIDs that were not part of the original
set(registry_session_uuids_list) - set(original_session_uuids_list)

{'1df10bfd-0f85-42ff-a873-36dfe05df77b', '2930a371-1187-4b98-adfc-19e095637e48'}

In [29]:

# Put the explanation in a summary sentance
print(
    f"Right away, we took out {anomalous_files_count} anomalous files because {nu.conjunctify_nouns(explanations_list)}"
    f" and repaired some multiple-triage files to add {len(set(registry_session_uuids_list) - set(original_session_uuids_list))} more sessions to the list."
)

Right away, we took out 19 anomalous files because 2 files had no user action taken, 5 files had no injury treatment being done, 6 files had no pulses being taken, 13 files had no patient accuracy rate above zero, and 8 files had no patients being engaged and repaired some multiple-triage files to add 2 more sessions to the list.


In [30]:

registry_session_uuids_count

328